<a href="https://colab.research.google.com/github/Aamir195/movie-mavia/blob/main/Hybrid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from surprise import Reader, Dataset, SVD
from surprise.model_selection import KFold
from surprise.model_selection.validation import cross_validate
import joblib

In [ ]:
pip install surprise

In [ ]:
main = pd.read_csv("movies_metadata.csv")
main.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.712900,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [ ]:
ratting = pd.read_csv("ratings.csv")
ratting.head()

,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546
4,1,1246,5.0,1425941556


In [ ]:
from datetime import datetime

In [ ]:
ratting["year"] = pd.to_datetime(ratting["timestamp"],errors='coerce').apply(lambda x: str(x).split('-')[0] if x != np.nan else np.nan)

In [ ]:
ratting.head()

,userId,movieId,rating,timestamp,year
0,1,110,1.0,1425941529,1970
1,1,147,4.5,1425942435,1970
2,1,858,5.0,1425941523,1970
3,1,1221,5.0,1425941546,1970
4,1,1246,5.0,1425941556,1970


In [ ]:
link = pd.read_csv("links_small.csv")
link.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [ ]:
main['overview'] = main['overview'].fillna('')
main['overview'].head() 

0    Led by Woody, Andy's toys live happily in his ...
1    When siblings Judy and Peter discover an encha...
2    A family wedding reignites the ancient feud be...
3    Cheated on, mistreated and stepped on, the wom...
4    Just when George Banks has recovered from his ...
Name: overview, dtype: object

In [ ]:
pd.DataFrame({'feature':main.dtypes.index, 'dtype':main.dtypes.values})

,feature,dtype
0,adult,bool
1,belongs_to_collection,object
2,budget,int64
3,genres,object
4,homepage,object
5,id,int64
6,imdb_id,object
7,original_language,object
8,original_title,object
9,overview,object


In [ ]:
# Check the datatype of "tmdbId" in links_small.csv
pd.DataFrame({'feature':link.dtypes.index, 'dtype':link.dtypes.values})

,feature,dtype
0,movieId,int64
1,imdbId,int64
2,tmdbId,float64


In [ ]:
# Convert float64 to int64
col=np.array(link['tmdbId'], np.int64)
link['tmdbId']=col

In [ ]:
# Merge the dataframes on column "tmdbId"
main.rename(columns={'id':'tmdbId'}, inplace=True)
main = pd.merge(main,link,on='tmdbId')
main.drop(['imdb_id'], axis=1, inplace=True)
main.head()

,adult,belongs_to_collection,budget,genres,homepage,tmdbId,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,movieId,imdbId
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,1,114709
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,2,113497
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.712900,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,3,113228
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,4,114885
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,5,113041


In [ ]:
main.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage',
       'tmdbId', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count', 'movieId', 'imdbId'],
      dtype='object')

In [ ]:
# Remove stop words and use TF-IDF vectorizer
tfidf = TfidfVectorizer(stop_words='english')
# Construct TF-IDF matrix
tfidf_matrix = tfidf.fit_transform(main['overview'])
tfidf_matrix.shape

(7310, 26804)

In [ ]:
# Compute cosine similarity
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
# Get corresponding indices of the movies
indices = pd.Series(main.index, index=main['original_title']).drop_duplicates()

In [ ]:
# Recommendation function
def recommend(title, cosine_sim=cosine_sim):
    
    # Get the index of the movie that matches the title
    idx = indices[title]
    
    # Get the pairwise similarity scores of all movies with the given movie
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Get the scores of the 15 most similar movies
    sim_scores = sim_scores[1:16]
    
    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]
    
    # Remove low-rated movies or outliers
    for i in movie_indices:
        pop = main.at[i,'vote_average']
        if pop<5 or pop>10:
            movie_indices.remove(i)

    # Return the most similar movies qualifying the 5.0 rating threshold
    return main[['original_title','vote_average',"genres","popularity"]].iloc[movie_indices]

In [ ]:
recommend('Sin Nombre')

,original_title,vote_average,genres,popularity
999,Touch of Evil,7.6,"[{'id': 18, 'name': 'Drama'}, {'id': 53, 'name...",9.665347
1630,Fandango,6.6,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",11.350645
3312,The Mexican,5.8,"[{'id': 28, 'name': 'Action'}, {'id': 35, 'nam...",9.958406
3266,Born in East L.A.,5.9,"[{'id': 35, 'name': 'Comedy'}]",2.118964
5933,The Jacket,6.8,"[{'id': 18, 'name': 'Drama'}, {'id': 9648, 'na...",12.092096
654,Lone Star,6.9,"[{'id': 18, 'name': 'Drama'}, {'id': 9648, 'na...",9.069963
3227,All the Pretty Horses,5.8,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",7.345133
65,From Dusk Till Dawn,6.9,"[{'id': 27, 'name': 'Horror'}, {'id': 28, 'nam...",15.339153
2952,Mad Max,6.6,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",11.412619
3243,Double Take,5.5,"[{'id': 12, 'name': 'Adventure'}, {'id': 18, '...",3.067916


In [ ]:
recommend("All the King's Men")

,original_title,vote_average,genres,popularity
999,Touch of Evil,7.6,"[{'id': 18, 'name': 'Drama'}, {'id': 53, 'name...",9.665347
2320,The Dark Half,5.4,"[{'id': 9648, 'name': 'Mystery'}, {'id': 27, '...",13.412785
2560,My Dog Skip,6.5,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",7.061185
4917,Mindwalk,6.5,"[{'id': 18, 'name': 'Drama'}]",0.122300
4889,Damage,6.7,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",6.384589
890,Rebel Without a Cause,7.6,"[{'id': 18, 'name': 'Drama'}]",10.968169
2773,La Bamba,6.8,"[{'id': 18, 'name': 'Drama'}, {'id': 10402, 'n...",6.476534
5315,Mean Streets,7.2,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",8.212399
2205,Frances,7.1,"[{'id': 18, 'name': 'Drama'}]",2.077869
343,Timecop,5.5,"[{'id': 53, 'name': 'Thriller'}, {'id': 878, '...",8.907074


In [ ]:
# -- User-focused recommender --

In [ ]:
reader = Reader() # Used to parse a file containing ratings
df = Dataset.load_from_df(ratting[['userId', 'movieId', 'rating']], reader)
kf = KFold(n_splits=5)
kf.split(df) # Split the data into folds

<generator object KFold.split at 0x7fa75d8e1b50>

In [ ]:
# Use Single Value Decomposition (SVD) for cross-validation and fitting
svd = SVD()
cross_validate(svd, df, measures=['RMSE', 'MAE'])


{'fit_time': (25.754799365997314,
  26.085610151290894,
  26.311344861984253,
  25.8182315826416,
  25.978811979293823),
 'test_mae': array([0.64655692, 0.64527726, 0.64285367, 0.64497725, 0.64360017]),
 'test_rmse': array([0.84730681, 0.84516248, 0.84355531, 0.84381357, 0.84504687]),
 'test_time': (1.3478610515594482,
  1.46120285987854,
  1.4016528129577637,
  1.5257594585418701,
  1.4326679706573486)}

In [ ]:
trainset = df.build_full_trainset()
svd.fit(trainset)

In [ ]:
ratting[ratting['userId'] == 14]

,userId,movieId,rating,timestamp,year
900,14,293,2.0,1488224496,1970
901,14,318,2.5,1488224438,1970
902,14,2571,4.5,1488224459,1970
903,14,2959,3.0,1488224441,1970
904,14,168252,5.0,1488224471,1970


In [ ]:
svd = joblib.load('collab.pkl')


In [ ]:
# Read the smaller links file again
link_df = pd.read_csv('links_small.csv')
col=np.array(link_df['tmdbId'], np.int64)
link_df['tmdbId']=col

# Merge movies_metadata.csv and links_small.csv files
link_df = link_df.merge(main[['title', 'tmdbId']], on='tmdbId').set_index('title')
link_index = link_df.set_index('tmdbId') # For label indexing

In [ ]:
# Recommendation function
def hybrid(userId, title):
    idx = indices[title]
    tmdbId = link_df.loc[title]['tmdbId'] # Get the corresponding tmdb_id
    
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    # sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31] # Scores of the 30 most similar movies
    movie_indices = [i[0] for i in sim_scores]
    
    movies = main.iloc[movie_indices][['title', 'vote_average', 'tmdbId']]
    movies['est'] = movies['tmdbId'].apply(lambda x: svd.predict(userId, link_index.loc[x]['movieId']).est) # Estimated prediction using svd
    movies = movies.sort_values('est', ascending=False) # Rank movies according to the predicted values
    movies.columns = ['Title', 'Vote Average', 'TMDb Id', 'Estimated Prediction']
    return movies.head(15) # Display top 15 similar movies

In [ ]:
# Recommendations for user with id 30
hybrid(30, 'Damage')

,Title,Vote Average,TMDb Id,Estimated Prediction
1435,Character,7.7,17139,4.412312
1501,All the King's Men,6.3,25430,4.210744
1820,Love Is the Devil: Study for a Portrait of Fra...,6.7,4975,4.122023
6043,Želary,7.3,19765,4.094805
4418,Confessions of a Dangerous Mind,6.6,4912,4.073534
891,A Streetcar Named Desire,7.6,702,4.053170
4543,What a Girl Wants,5.8,10735,4.015878
3317,Blow Dry,5.8,17203,4.015566
4642,From the Terrace,5.8,42541,4.008745
2291,The Buddy Holly Story,7.5,24153,3.993130


In [ ]:
# Recommendations for user with id 500
hybrid(500, 'Four Weddings and a Funeral')

,Title,Vote Average,TMDb Id,Estimated Prediction
4981,A Passage to India,6.8,15927,3.797928
6604,Seven Up!,7.5,51863,3.704208
739,Gone with the Wind,7.7,770,3.606665
6714,Death at a Funeral,6.9,2196,3.555228
3302,In the Mood for Love,7.8,843,3.434292
4458,Missing,7.2,15600,3.434172
777,Little Lord Fauntleroy,6.6,23114,3.401074
6164,Wedding Crashers,6.4,9522,3.361411
1516,Chariots of Fire,6.9,9443,3.342748
5247,Gunga Din,6.6,24965,3.330595


In [ ]:
hybrid(132425, 'Clockwise')

,Title,Vote Average,TMDb Id,Estimated Prediction
4998,Diabolique,7.9,827,4.427698
535,Fargo,7.7,275,4.204225
2755,The Color of Paradise,7.1,17078,4.128075
2079,Election,6.7,9451,3.996648
6582,The History Boys,6.6,8618,3.972908
5916,Evil,6.9,11197,3.829694
997,Dead Poets Society,8.1,207,3.783795
6482,Step Up,6.7,9762,3.714713
6656,The Inglorious Bastards,5.9,22311,3.708004
3806,Harry Potter and the Philosopher's Stone,7.5,671,3.701112
